In [ ]:
import warnings
warnings.filterwarnings(action='once')
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import datetime
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pickle
%load_ext autoreload
%autoreload 2
%matplotlib inline
from exp.misc import *
from exp.ProcessData import *
from exp.PytorchModels import *
from exp.LearnerClass import *
import torch
import torch.nn as nn
import torch.utils.data as D
import torch.nn.functional as F
import copy
from torchvision import transforms
import PIL.Image
from sklearn.metrics import roc_auc_score
import torchvision.transforms.functional as TF
from types import MethodType
import sandesh
import pydicom

## Set Parameters

In [ ]:
params=json_to_parameters('config.json')
num_folds=5
folds=[0]
device = device_by_name('Tesla')
model_type='tf_efficientnet_b5_ns' #'se_resnext101_32x4d'
loss_int = True
pool=False
dropout=0.5
wso_requires_grad=True
folds=[1]
seed_add=22222
model_version=1
num_tta=0
output_size=11
torch.cuda.set_device(device)

In [ ]:
df = pd.read_csv(params.path.data+'full_train.csv')

In [ ]:
transform=transforms.Compose([RandomResizedCropTransform(scale=(0.7, 1.3), ratio=(0.7, 1.3)),
                              RotateTransform(25),
                              RandomFlipTransform(0.5,0.5),
                              RandomChangeMeanStdTransform(30,0.1),
                              CutoutTransform(0.5,0.2),
                              CutoutTransform(0.25,0.3),
                              ResizeTransform(512,512)])

transform_val=transforms.Compose([ResizeTransform(512,512)])


In [ ]:
model_type='tf_efficientnet_b5_ns'
name_tamplate='image_{}'
image_reader=ImageReader(params.path.train)
ds0=ImageDataset(image_reader,df,transform=transform_val)
ds=ImageDataset(image_reader,df,transform=transform)
for SEED in [220]:
    for fold in folds: 
        torch.manual_seed(SEED+fold+seed_add)
        np.random.seed(SEED+fold+seed_add)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        model = get_model(model_type,output_size=output_size, feature_size =256,amp=True,pool=pool,
                          dropout=dropout,wso_requires_grad=wso_requires_grad).to(device)
        name=params.model_format.format(model_type,f'image_{SEED}',model_version,fold)
        print (name)
        learner = Learner(model,None,loss_func=None,name=name,scheduler=None,device=device)
        learner.load_model(params.path.models)
#        learner.init_amp()
        learner.model.last_linear=Noop()
        def new_get_x(self,batch):
            return batch[:1] 
        learner.get_x=MethodType(new_get_x, learner)
#         featuress=[learner.predict(ds0,batch_size=64,num_workers=20,do_eval=True)]
        featuress=[learner.predict(ds0,batch_size=64,num_workers=20,do_eval=True)]
        sandesh.send({'name':f'finished 1st {SEED} {fold}'})
        for i in notebook.tqdm(range(num_tta)):
#                 learner.model=learner.model.train()
            features=learner.predict(ds,batch_size=64,num_workers=20,do_eval=True)
            featuress.append(features)
            sandesh.send({'name':f'finished {i+1} {SEED} {fold}'})

        features=np.stack(featuress,0)
        with open(params.path.features+(name.split('.')[0]+'.pkl'),'wb') as f:
            pickle.dump(features,f,protocol=4)
        sandesh.send({'name':f'saved {features.shape} {SEED} {fold}'})
